In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
corona_df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
corona_df.head(5)

In [ ]:
corona_df.tail(5)

In [ ]:
corona_df.isnull().sum()

In [ ]:
corona_df.duplicated().sum()

In [ ]:
corona_df.info()

In [ ]:
corona_df = corona_df.drop(['Sno','Time','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)

In [ ]:
corona_df.head()

In [ ]:
corona_df = corona_df.rename(columns={'State/UnionTerritory':'States','Cured':'Recovered'})

In [ ]:
corona_df.head()

In [ ]:
corona_df['Active'] = corona_df['Confirmed'] - corona_df['Recovered'] - corona_df['Deaths']

In [ ]:
corona_df = corona_df.sort_values(['Date', 'States']).reset_index(drop=True)

In [ ]:
corona_df.head()

In [ ]:
corona_df['Date'] = pd.to_datetime(corona_df['Date'])

In [ ]:
corona_df.head()

In [ ]:
corona_df.info()

In [ ]:
india_cases = corona_df[corona_df['Date'] == corona_df['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["States"]
india_cases = india_cases.drop(['States', 'Date'], axis=1)

In [ ]:
india_cases.head()

In [ ]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='RdPu',axis=1)

In [ ]:
india_cases.sort_values('Confirmed', ascending= False).style\
    .background_gradient(cmap='BrBG_r', subset=['Confirmed'])\
    .background_gradient(cmap='YlGn_r', subset=["Deaths"])\
    .background_gradient(cmap='YlOrBr',subset=['Recovered'])\
    .background_gradient(cmap='BuPu', subset=['Active'])

In [ ]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
import plotly.express as px

In [ ]:
top_10_indian_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_indian_confirmed_states,top_10_indian_confirmed_states.Confirmed, top_10_indian_confirmed_states.index,
                     'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','Teal')

In [ ]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index, 'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Red')

In [ ]:
top_10_indian_recovered_states = india_cases.sort_values('Recovered', ascending=False)[:10]
horizontal_bar_chart(top_10_indian_recovered_states, top_10_indian_recovered_states.Recovered, top_10_indian_recovered_states.index,
                    'Top 10 States (Recovered Cases)', 'Number of Recovered cases (in Thousands)', 'States Name', 'Purple')

In [ ]:
top_10_india_active_states = india_cases.sort_values('Active', ascending=False)[:10]
horizontal_bar_chart(top_10_india_active_states, top_10_india_active_states.Active, top_10_india_active_states.index,
                     'Top 10 States (Active Cases)', 'Number of Active cases (in Thousands)', 'States Name', 'Green')

In [ ]:
import geopandas as gpd
fp = '../input/india-2020-gisdata/Igismap/Indian_States.shp'
map_df = gpd.read_file(fp)

# change state name to match in both files -- I manually checked few
map_df['st_nm'].iloc[0]  = 'Andaman and Nicobar Islands'
map_df['st_nm'].iloc[12] = 'Jammu and Kashmir'
map_df['st_nm'].iloc[6] = 'Dadara and Nagar Havelli'
map_df['st_nm'].iloc[23] = 'Delhi'

In [ ]:
merged = map_df.set_index('st_nm').join(corona_df.set_index('States'))

#fill NaN values with Zero

merged[['Confirmed', 'Recovered', 'Deaths']] = merged[['Confirmed', 'Recovered', 'Deaths']].fillna(0).astype('int')

In [ ]:
fig, ax = plt.subplots(1, figsize=(23, 19))
ax.axis('on')
ax.set_title('Total COVID-19 cases in India', fontdict={'fontsize': '35', 'fontweight' : '5'})

# plot the figure
merged.plot(column='Confirmed', cmap='Teals', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)